<a href="https://colab.research.google.com/github/tanjinadnanabir/Mental-Health-AI-1.0/blob/main/mental_health_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# V 1.0

In [ ]:
!pip install -q transformers accelerate sentence-transformers faiss-gpu-cu12 sqlite-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
import faiss
import torch
import sqlite3
import numpy as np

from transformers import pipeline
from sentence_transformers import SentenceTransformer

device = 0 if torch.cuda.is_available() else -1
print("Using GPU" if device == 0 else "Using CPU")

Using GPU


In [ ]:
emotion_classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    device=device,
    return_all_scores=True
)

def detect_emotion(text):
    scores = emotion_classifier(text)[0]
    scores = sorted(scores, key=lambda x: x["score"], reverse=True)
    return scores[0]["label"], scores

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
CRISIS_KEYWORDS = [
    "suicide", "kill myself", "end my life",
    "self harm", "cut myself", "want to die",
    "no reason to live", "better off dead"
]

def detect_crisis(text, emotion):
    text = text.lower()
    keyword_hit = any(k in text for k in CRISIS_KEYWORDS)
    risky_emotions = ["sadness", "fear", "anger"]
    return keyword_hit and emotion in risky_emotions

In [ ]:
conn = sqlite3.connect("memory.db")
conn.execute("""
CREATE TABLE IF NOT EXISTS memory (
    user_id TEXT,
    key TEXT,
    value TEXT
)
""")
conn.commit()

def save_memory(user_id, key, value):
    conn.execute("INSERT INTO memory VALUES (?, ?, ?)", (user_id, key, value))
    conn.commit()

def load_memory(user_id):
    cur = conn.execute("SELECT key, value FROM memory WHERE user_id=?", (user_id,))
    return dict(cur.fetchall())

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

knowledge_base = [
    "Deep breathing calms the nervous system",
    "CBT helps identify negative thought patterns",
    "Grounding techniques reduce panic",
    "Talking to trusted people reduces emotional burden",
    "Therapy is effective for anxiety, depression, and PTSD",
    "Regular sleep improves mental health",
    "Exercise releases endorphins that improve mood"
]

embeddings = embedder.encode(knowledge_base, convert_to_numpy=True)

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
res = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)
gpu_index.add(embeddings)

In [ ]:
def retrieve_context(query, k=3):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    distances, indices = gpu_index.search(q_emb, k)
    return [knowledge_base[i] for i in indices[0]]

In [ ]:
def empathy_prefix(emotion):
    responses = {
        "sadness": "I'm really sorry you're feeling this way. You're not alone.",
        "fear": "That sounds overwhelming. Thank you for trusting me.",
        "anger": "It seems like there's a lot of pain and frustration.",
        "joy": "I'm glad to hear some positivity."
    }
    return responses.get(emotion, "I'm here with you.")

In [ ]:
def route_agent(crisis, emotion):
    if crisis:
        return "crisis"
    elif emotion in ["sadness", "fear"]:
        return "clinical"
    else:
        return "general"

In [ ]:
def crisis_agent():
    return (
        "🚨 I'm really concerned about your safety.\n\n"
        "If you're in immediate danger, please seek help NOW:\n"
        "🇧🇩 Bangladesh: Kaan Pete Roi – 09639-777-777\n"
        "🌍 Find local helplines: findahelpline.com\n\n"
        "You matter. Help is available."
    )

def clinical_agent():
    return (
        "What you're experiencing may be linked to anxiety or depression.\n"
        "Would you like coping strategies or professional support?"
    )

def general_agent():
    return "I'm here to listen. Can you share more about what's going on?"

In [ ]:
def coping_strategies(emotion):
    strategies = {
        "sadness": ["Journaling", "Gentle walk", "Talk to a trusted person"],
        "fear": ["Box breathing", "5-4-3-2-1 grounding"],
        "anger": ["Cold water splash", "Push-ups or walking"]
    }
    return strategies.get(emotion, ["Mindful breathing"])

In [ ]:
def recommend_resources():
    return [
        "WHO Mental Health Self-Help Guide",
        "Free CBT worksheets (online)",
        "Local mental health clinics"
    ]

In [ ]:
def book_appointment(user_id):
    return "✅ Appointment request saved. A clinician will contact you."

In [ ]:
def mental_health_agent(user_id, user_input):
    emotion, scores = detect_emotion(user_input)
    crisis = detect_crisis(user_input, emotion)
    agent_type = route_agent(crisis, emotion)

    save_memory(user_id, "last_emotion", emotion)
    rag_context = retrieve_context(user_input)

    prefix = empathy_prefix(emotion)

    if agent_type == "crisis":
        reply = crisis_agent()
    elif agent_type == "clinical":
        reply = clinical_agent()
    else:
        reply = general_agent()

    return {
        "emotion": emotion,
        "agent": agent_type,
        "response": prefix + "\n\n" + reply,
        "coping_strategies": coping_strategies(emotion),
        "resources": recommend_resources(),
        "rag_context": rag_context
    }

In [ ]:
output = mental_health_agent(
    user_id="user_001",
    user_input="I feel completely hopeless and exhausted. I don't see a future."
)

for k, v in output.items():
    print(f"\n{k.upper()}:\n{v}")


EMOTION:
sadness

AGENT:
clinical

RESPONSE:
I'm really sorry you're feeling this way. You're not alone.

What you're experiencing may be linked to anxiety or depression.
Would you like coping strategies or professional support?

COPING_STRATEGIES:
['Journaling', 'Gentle walk', 'Talk to a trusted person']

RESOURCES:
['WHO Mental Health Self-Help Guide', 'Free CBT worksheets (online)', 'Local mental health clinics']

RAG_CONTEXT:
['Exercise releases endorphins that improve mood', 'Regular sleep improves mental health', 'Therapy is effective for anxiety, depression, and PTSD']


# V 1.1

In [ ]:
!pip install -q transformers accelerate bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.1 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using:", device)

Using: cuda


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Define quantization config as recommended in the warning
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config, # Use quantization_config instead of load_in_4bit
    torch_dtype=torch.float16
)

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

Device set to use cuda:0


In [ ]:
def build_empathetic_prompt(
    user_input,
    emotion,
    rag_context,
    crisis=False
):
    system_prompt = f"""
You are a compassionate mental health support assistant.
You are NOT a doctor.
You must NEVER encourage self-harm or suicide.
If crisis is detected, encourage seeking immediate human help.

User emotion: {emotion}
Crisis detected: {crisis}

Relevant mental health knowledge:
{chr(10).join(rag_context)}

Respond with empathy, validation, and gentle support.
Avoid medical diagnosis.
Be calm, respectful, and human-like.
"""

    user_prompt = f"""
User says:
"{user_input}"

Your response:
"""

    return system_prompt + user_prompt

In [ ]:
def enforce_crisis_safety(text):
    banned_phrases = [
        "you should kill yourself",
        "end your life",
        "no reason to live"
    ]
    for phrase in banned_phrases:
        if phrase in text.lower():
            return (
                "I'm really glad you reached out. "
                "You deserve help and care. "
                "Please contact a trusted person or a crisis hotline immediately."
            )
    return text

In [ ]:
def generate_empathetic_response(
    user_input,
    emotion,
    rag_context,
    crisis=False
):
    prompt = build_empathetic_prompt(
        user_input=user_input,
        emotion=emotion,
        rag_context=rag_context,
        crisis=crisis
    )

    output = generator(prompt)[0]["generated_text"]

    # Remove prompt echo
    response = output.split("Your response:")[-1].strip()

    # Safety enforcement
    response = enforce_crisis_safety(response)

    return response

In [ ]:
def mental_health_agent_llm(user_id, user_input):
    emotion, _ = detect_emotion(user_input)
    crisis = detect_crisis(user_input, emotion)
    agent_type = route_agent(crisis, emotion)

    save_memory(user_id, "last_emotion", emotion)

    rag_context = retrieve_context(user_input)

    llm_response = generate_empathetic_response(
        user_input=user_input,
        emotion=emotion,
        rag_context=rag_context,
        crisis=crisis
    )

    return {
        "emotion": emotion,
        "agent": agent_type,
        "response": llm_response,
        "coping_strategies": coping_strategies(emotion),
        "resources": recommend_resources(),
        "rag_context": rag_context
    }

In [ ]:
output = mental_health_agent_llm(
    user_id="user_001",
    user_input="I've been feeling constantly anxious and unable to focus on anything for more than a few minutes at a time. I can't seem to get anything done, and I'm starting to feel like I'm failing everyone who's relying on me."
)

for k, v in output.items():
    print(f"\n{k.upper()}:\n{v}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



EMOTION:
fear

AGENT:
clinical

RESPONSE:
"I'm really sorry that you're feeling this way right now. It's completely normal to experience anxiety and difficulty focusing from time to time, especially when we have a lot on our plates. But it sounds like things might be getting a bit overwhelming for you lately.

Have you tried any grounding techniques to help manage your anxiety? Simple activities like deep breathing, progressive muscle relaxation, or even just focusing on an object in your environment can help bring your attention back to the present moment and reduce feelings of panic.

It's also important to remember that seeking professional help, such as therapy, can be incredibly beneficial for managing anxiety, depression, and other mental health challenges. A mental health professional can provide you with tools and strategies to better understand and cope with your emotions, as well as offer a safe and supportive space to talk about what's going on in your life.

In the meantim

# V 2.0

In [ ]:
!pip install pymilvus sentence-transformers transformers torch chromadb numpy pandas scikit-learn textblob vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 13.9 MB/s eta 0:00:00
